<a href="https://colab.research.google.com/github/PriyaBansal68/ML-Project-winter-2022/blob/main/Football_Logistic_Neural_XG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools

%matplotlib inline
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
#A random forest is a meta estimator that fits a number of decision tree classifiers
#on various sub-samples of the dataset and use averaging to improve the predictive
#accuracy and control over-fitting.
from sklearn.ensemble import RandomForestClassifier
#a discriminative classifier formally defined by a separating hyperplane.
from sklearn.svm import SVC
#displayd data
from IPython.display import display
%matplotlib inline
from sklearn.metrics import log_loss,accuracy_score,roc_auc_score,f1_score
from sklearn.metrics import classification_report

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Football/train.csv')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.columns.tolist()

In [ ]:
data=df.drop(['match_date','home_team_history_match_date_1','league_name',
 'home_team_history_match_date_2',
 'home_team_history_match_date_3',
 'home_team_history_match_date_4',
 'home_team_history_match_date_5',
 'home_team_history_match_date_6',
 'home_team_history_match_date_7',
 'home_team_history_match_date_8',
 'home_team_history_match_date_9',
 'home_team_history_match_date_10', 
 'away_team_history_match_date_1',
 'away_team_history_match_date_2',
 'away_team_history_match_date_3',
 'away_team_history_match_date_4',
 'away_team_history_match_date_5',
 'away_team_history_match_date_6',
 'away_team_history_match_date_7',
 'away_team_history_match_date_8',
 'away_team_history_match_date_9','away_team_history_match_date_10'], axis=1)

In [ ]:
data

In [ ]:
#win rate for the home team?

# Total number of matches.
n_matches = data.shape[0]

# Calculate number of features. -1 because we are saving one as the target variable (win/lose/draw)
n_features = data.shape[1] - 1

# Calculate matches won by home team.
n_homewins = len(data[data.target == 'home'])

# Calculate win rate for home team.
win_rate = (float(n_homewins) / (n_matches)) * 100

# Print the results
print ("Total number of matches: {}".format(n_matches))
print ("Number of features: {}".format(n_features))
print ("Number of matches won by home team: {}".format(n_homewins))
print ("Win rate of home team: {:.2f}%".format(win_rate))

In [ ]:
pip install category_encoders

In [ ]:
data=data.dropna()
data

In [ ]:
from sklearn.model_selection import train_test_split
import category_encoders as ce
# Shuffle and split the dataset into training and testing set.


In [ ]:
data['target'] = data['target'].replace(['away','draw','home'],[0, 1, 2])

In [ ]:
encoder = ce.OrdinalEncoder(cols=['home_team_name' , 'away_team_name', 'is_cup'])
data = encoder.fit_transform(data)

In [ ]:
data.dtypes

In [ ]:
data

In [ ]:
#plt.barh(data.columns, rf.feature_importances_)
#rf.feature_importances_


In [ ]:
def add_streak(data):
    h_t_goal = "home_team_history_goal_"
    h_t_opp_goal = "home_team_history_opponent_goal_"
    a_t_goal = "away_team_history_goal_"
    a_t_opp_goal = "away_team_history_opponent_goal_"
    df["away_streak"] = np.nan
    df["home_streak"] = np.nan
    def apply_streaks(x):
        home_streak = 0
        away_streak = 0
        home_streak_over = False
        away_streak_over = False
        
        if  x[h_t_goal+str(1)] > x[h_t_opp_goal+str(1)]:
            home_win_streak = True
        else:
            home_win_streak = False
            
        if  x[a_t_goal+str(1)] > x[a_t_opp_goal+str(1)]:
            away_win_streak = True
        else:
            away_win_streak = False
            
        def check_streak(streak,streak_over,win_streak,team_goal,opp_goal):
            if not streak_over and win_streak and team_goal > opp_goal:
                streak+=1
            elif not streak_over and not win_streak and team_goal < opp_goal:
                streak-=1
            elif not streak_over and team_goal == opp_goal:
                pass
            else:
                streak_over = True
            return streak, streak_over
                
        for i in range(1,11):
            if not home_streak_over:
                home_streak, home_streak_over = check_streak(home_streak,home_streak_over, home_win_streak, x[h_t_goal+str(i)],x[h_t_opp_goal+str(i)])
            if not away_streak_over:
                away_streak, away_streak_over = check_streak(away_streak,away_streak_over, away_win_streak, x[a_t_goal+str(i)],x[a_t_opp_goal+str(i)])
            
        return pd.Series([home_streak,away_streak],index=['home_streak','away_streak'])

    
    df[['home_streak','away_streak']] = df.apply(lambda x: apply_streaks(x), axis=1)
    return data

In [ ]:
data = add_streak(data)

In [ ]:
for i in data.columns:
    if data[i].dtype == 'object':
        data = data.drop([i],axis=1)
        data = data.drop([i],axis=1)
    if "is_cup" in i or "history_is_play_home" in i:
        try:
            data = data.drop([i],axis=1)
            #X_test = X_test.drop([i],axis=1)
        except:
            continue

for i in data.columns:
    if 'rating' not in i and 'goal':
        data[i] = data[i].fillna(-1)
        #X_test[i] = X_test[i].fillna(-1)
    else:
        data[i] = data[i].fillna(data[i].mode())
        #X_test[i] = X_test[i].fillna(X_train[i].mode())

In [ ]:
data

In [ ]:
X = data.drop(['target'], axis = 1)
y=data['target']

In [ ]:
X

In [ ]:
y

In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
encoder = LabelEncoder()
encoder.fit(y)
encoded_Y = encoder.transform(y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
y_cat = to_categorical(y)

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_scaled = pd.DataFrame(sc.fit_transform(X))

In [ ]:
X_scaled


In [ ]:
y_cat

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, 
                                                    test_size = 0.2,
                                                    random_state = 42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100)
rf.fit(X_train, y_train)

In [ ]:
from keras.layers import Dense,Softmax, Dropout
from tensorflow.keras.optimizers import Adam, Adadelta, RMSprop

modelN = Sequential()
modelN.add(Dense(60, input_shape = (127,), activation = "relu"))
modelN.add(Dense(15, activation = "relu"))
modelN.add(Dropout(0.2))
modelN.add(Dense(3, activation = "softmax"))
modelN.compile(Adam(lr = 0.01), "sparse_categorical_crossentropy", metrics = ["accuracy"])
modelN.summary()

In [ ]:
modelN.fit(X_train, y_train, verbose=1, epochs=10)

In [ ]:

#y_pred_class = modelN.predict_classes(X_test)
predict_x=modelN.predict(X_test) 
y_pred_class=np.argmax(predict_x,axis=1)
from sklearn.metrics import confusion_matrix
y_pred = modelN.predict(X_test)
y_test_class = np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import classification_report
confusion_matrix(y_test_class, y_pred_class)
print(classification_report(y_test_class, y_pred_class))
#print('Log loss score with  : {0:0.4f}'. format(log_loss(y_test_class,y_pred_class)))
#print('Model accuracy score with : {0:0.4f}'. format(accuracy_score(y_test, predict_x)))
#print('Model ROC with : {0:0.4f}'. format(roc_auc_score(y_test_class, y_pred_class,multi_class='ovr',average='weighted')))

In [ ]:
model_NN = Sequential([
    layers.Input(127),
    layers.Dense(60, activation = 'relu'),
    layers.Dense(30,activation='relu'),
    layers.Dense(10, activation = 'relu'),
    layers.Dense(3)
])

In [ ]:
model_NN.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model_NN.summary()

In [ ]:
#y_train1=pd.DataFrame(data=y_train)
df.dtypes

In [ ]:
history = model_NN.fit(X_train, pd.DataFrame(data=y_train), batch_size=11000, epochs=500, verbose=1)

In [ ]:
plt.plot(history.history['loss'])

plt.legend();

In [ ]:
plt.plot(history.history['accuracy'])

plt.legend();

In [ ]:
from sklearn.metrics import matthews_corrcoef
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import log_loss,accuracy_score,roc_auc_score,f1_score
from sklearn.metrics import classification_report

In [ ]:
predictions = model_NN.predict(X_test)

y_predN=np.argmax(predictions,axis=1)
y_pred_accN = model_NN.predict(X_test)
#y_test = np.argmax(y_test, axis=1)

print(y_test.values)
print(y_predN)



In [ ]:
scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

NN_Predictions=model_NN.predict(scaled_X_test)
#ypred = (NN_Predictions > 0.5).astype(int)

ypred=np.argmax(NN_Predictions,axis=1)


print('Log loss score with  : {0:0.4f}'. format(log_loss(y_test.values,ypred)))
#print('Model accuracy score with : {0:0.4f}'. format(matthews_corrcoef(y_test,ypred)))
print('Model roc score with : {0:0.4f}'. format(roc_auc_score(y_test,ypred,multi_class='ovr',average='weighted')))
print('Model f1 score with : {0:0.4f}'. format(f1_score(y_test, ypred)))
  

In [ ]:
from xgboost import XGBClassifier
#from pandas import MultiIndex, Int64Index


best_hyperparams = {'colsample_bytree': 0.9080289013373745, 'gamma': 1.2419885617337658, 'max_depth': 8.0, 'min_child_weight': 4.0, 
'n_estimators': 238.0, 'reg_alpha': 41.0, 'reg_lambda': 0.05050398016408508}

rfc = XGBClassifier(n_estimators=int(best_hyperparams['n_estimators']), max_depth = int(best_hyperparams['max_depth']), 
gamma = best_hyperparams['gamma'],
                    reg_alpha = int(best_hyperparams['reg_alpha']),min_child_weight=int(best_hyperparams['min_child_weight']),
                    colsample_bytree=int(best_hyperparams['colsample_bytree']))
rfc.fit(X_train, y_train)
y_pred = rfc.predict_proba(X_test)
y_pred_acc = rfc.predict(X_test)

print(y_test.values)
print(y_pred)
print('Log loss score with  : {0:0.4f}'. format(log_loss(y_test.values,y_pred)))
print('Model accuracy score with : {0:0.4f}'. format(accuracy_score(y_test, y_pred_acc)))
print('Model ROC with : {0:0.4f}'. format(roc_auc_score(y_test.values, y_pred,multi_class='ovr',average='weighted')))

In [ ]:
from sklearn.linear_model import LogisticRegression
model_LR=Pipeline([('scaler', MinMaxScaler()), ('Logmodel', LogisticRegression(C=0.001,penalty='l2',solver='saga'))])
#model_LR = LogisticRegression(C=0.001,penalty='l2',solver='saga')
model_LR.fit(X_train,y_train)

In [ ]:
y_pred_acc1 = model_LR.predict(X_test)
y_pred2 = model_LR.predict_proba(X_test)

print('Log loss score with  : {0:0.4f}'. format(log_loss(y_test.values,predictions)))
print('Model accuracy score with : {0:0.4f}'. format(accuracy_score(y_test, y_pred_acc1)))
print('Model ROC with : {0:0.4f}'. format(roc_auc_score(y_test, y_pred2,multi_class="ovr",average='weighted')))